# 0. Librerías

In [13]:
# Genérica
# -----------------------------------------------------------------------------
from time import time
import session_info


# Tratamientos datos
# -----------------------------------------------------------------------------
import pandas as pd
import numpy as np


# category encoders
# -----------------------------------------------------------------------------
from category_encoders import OrdinalEncoder


# scikit-learn
# -----------------------------------------------------------------------------
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


# LightGBM
# -----------------------------------------------------------------------------
import lightgbm as lgb


# shapash
# -----------------------------------------------------------------------------
from shapash.explainer.smart_explainer import SmartExplainer


# Gráficos
# -----------------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt


session_info.show(html=False)

-----
category_encoders   2.6.1
lightgbm            3.3.5
matplotlib          3.7.2
numpy               1.23.5
pandas              1.5.3
seaborn             0.12.2
session_info        1.0.0
shapash             2.2.0
sklearn             1.3.0
-----
IPython             8.14.0
jupyter_client      8.3.0
jupyter_core        5.3.1
notebook            6.5.4
-----
Python 3.10.11 (main, May 16 2023, 00:28:57) [GCC 11.2.0]
Linux-5.15.0-91-generic-x86_64-with-glibc2.31
-----
Session information updated at 2023-12-28 12:40


# 1. Entrenar modelo

In [2]:
# Ruta
path = './data/tablon-sell-in-4.csv'


# Cargar datos
df = pd.read_csv(
    path,
    sep=';',
    dtype={
        'Epoca_Covid': 'category',
        'Anio': 'category',
        'Trimestre': 'category',
        'Mes': 'category',
        'Cliente': 'category',
        'Tipo_Cliente': 'category',
        'Distribuidor_MSM': 'category',
        'Codigo_Postal': 'category',
        'Area_Dist': 'category',
        'Zona': 'category',
        'ID_CCAA': 'category',
        'Comunidad_Autonoma': 'category',
        'ID_Provincia': 'category',
        'Provincia': 'category',
        'Gama': 'category',
        'Formato_1': 'category',
        'Formato_2': 'category'
    })


# Castear tipo de dato
fcols = df.select_dtypes('float').columns
icols = df.select_dtypes('integer').columns

df[fcols] = df[fcols].apply(pd.to_numeric, downcast='float')
df[icols] = df[icols].apply(pd.to_numeric, downcast='integer')


# Eliminar año 2020
filtro = (df.Anio != '2020')
# filtro = (df.Anio == '2022') & (df.Tipo_Cliente == 'CON')
# filtro = (df.Cliente == '542516')
df = df[filtro]


df.head()

,Fecha,Epoca_Covid,Anio,Trimestre,Mes,Cliente,Tipo_Cliente,Distribuidor_MSM,Codigo_Postal,Area_Dist,...,IPRI,Temperatura_Minima,Temperatura_Media,Temperatura_Maxima,Precipitacion,Sell_In_3,Sell_In_2,Sell_In_1,Sell_In,STRATIO_CREATED_DATE
0,201610,pre-covid,2016,T4,10,534564,DHC,0,30004,ZZ,...,102.449997,15.11,20.379999,25.66,27.65,0.32,0.32,0.32,0.32,2023-12-14T16:00:48.156Z
1,201610,pre-covid,2016,T4,10,534564,DHC,0,30004,ZZ,...,102.449997,15.11,20.379999,25.66,27.65,0.00,0.00,0.00,0.00,2023-12-14T16:00:48.156Z
2,201610,pre-covid,2016,T4,10,534564,DHC,0,30004,ZZ,...,102.449997,15.11,20.379999,25.66,27.65,3.00,3.00,3.00,3.00,2023-12-14T16:00:48.156Z
3,201610,pre-covid,2016,T4,10,534564,DHC,0,30004,ZZ,...,102.449997,15.11,20.379999,25.66,27.65,0.06,0.06,0.06,0.06,2023-12-14T16:00:48.156Z
4,201610,pre-covid,2016,T4,10,534564,DHC,0,30004,ZZ,...,102.449997,15.11,20.379999,25.66,27.65,0.16,0.16,0.16,0.16,2023-12-14T16:00:48.156Z


In [3]:
# Eliminar columnas
df = df.drop(columns=[
    'Fecha',                      # Mucha precisión, no generaliza el modelo
    'Anio',                       # Mucha precisión, no generaliza el modelo
    'Cliente',                    # Mucha precisión, no generaliza el modelo
    'Codigo_Postal',              # Mucha precisión, no generaliza el modelo
    'Frontur',                    # Colinear
    'Temperatura_Minima',         # Colinear
    'Temperatura_Maxima',         # Colinear
    'Movimientos_Entrada',        # Colinear
    'Movimientos_Salida',         # Colinear
    'Volumen_Entregado_Perfecto', # Colinear
    'Sell_In_3',                  # Colinear
    'Sell_In_2',                  # Colinear
    'Comunidad_Autonoma',         # Colinear
    'Provincia',                  # Colinear
    'Numerica',                   # Colinear
    'ID_CCAA',                    # Poco feature importance
    'Trimestre',                  # Poco feature importance
    'Distribuidor_MSM',           # Poco feature importance
    'Zona',                       # Poco feature importance
    # 'Num_PdV_CI',                 # Poco feature importance
    # 'Num_PdV_RU',                 # Poco feature importance
    # 'Num_PdV_TU',                 # Poco feature importance
    'Impactos_OOH',               # Poco feature importance
    'Impresiones_Meta',           # Poco feature importance
    'Impresiones_Pint',           # Poco feature importance
    'Impresiones_Digi',           # Poco feature importance
    'Distancia_CdP',              # Poco feature importance
    'Nivel_Servicio',             # Poco feature importance
    'Num_Festivos',               # Poco feature importance
    'Num_Festivos_AI',            # Poco feature importance
    'Movimientos_Neto',           # Poco feature importance
    # 'IPC',                        # Poco feature importance
    'IPI',                        # Poco feature importance
    # 'Temperatura_Media',          # Poco feature importance
    # 'Precipitacion',              # Poco feature importance
    'STRATIO_CREATED_DATE'        # Auditoría
])

# Eliminar registros con NaN
df = df.dropna()

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2275963 entries, 0 to 2644110
Data columns (total 28 columns):
 #   Column               Dtype   
---  ------               -----   
 0   Epoca_Covid          category
 1   Mes                  category
 2   Tipo_Cliente         category
 3   Area_Dist            category
 4   ID_Provincia         category
 5   Gama                 category
 6   Formato_1            category
 7   Formato_2            category
 8   Num_PdV_CI           float32 
 9   Num_PdV_RU           float32 
 10  Num_PdV_TU           float32 
 11  Impactos_TV          float64 
 12  Sell_In_PP           float64 
 13  Marketing_Nacional   float64 
 14  Trade_Marketing      float64 
 15  Marketing_PLV        float64 
 16  Dispersion_PdV       float32 
 17  Volumen_Solicitado   float32 
 18  Tarifa_Eur_Hl        float64 
 19  Margen_Distribuidor  float64 
 20  Egatur               float32 
 21  Familitur            float32 
 22  IPC                  float32 
 23  IPRI   

In [5]:
# Variable dependiente
target = 'Sell_In'

# Variables independientes
features = list(df.columns)
features.remove(target)

# Mostrar
print(f'Variable dependiente: {target}')
print(30*'-')
print(f'Variable independiente: {features}')

Variable dependiente: Sell_In
------------------------------
Variable independiente: ['Epoca_Covid', 'Mes', 'Tipo_Cliente', 'Area_Dist', 'ID_Provincia', 'Gama', 'Formato_1', 'Formato_2', 'Num_PdV_CI', 'Num_PdV_RU', 'Num_PdV_TU', 'Impactos_TV', 'Sell_In_PP', 'Marketing_Nacional', 'Trade_Marketing', 'Marketing_PLV', 'Dispersion_PdV', 'Volumen_Solicitado', 'Tarifa_Eur_Hl', 'Margen_Distribuidor', 'Egatur', 'Familitur', 'IPC', 'IPRI', 'Temperatura_Media', 'Precipitacion', 'Sell_In_1']


In [6]:
# Seed
seed = 123

# Separar conjunto de test y entrenamiento
train, test = train_test_split(df, train_size=0.95, test_size=0.05, random_state=seed)

In [7]:
# Instanciar encoder
categorical_features = [col for col in features if df[col].dtype == 'object']

encoder = OrdinalEncoder(
    cols=categorical_features,
    handle_unknown='ignore',
    return_df=True
    )

# Instanciar modelo
max_depth = 8

mlgb = lgb.LGBMRegressor(
    boosting_type = 'gbdt', # Gradient Boosting Decision Tree
    learning_rate = 0.1, # Boosting learning rate (default value)
    n_estimators = 1000, # Número de árboles (originalmente 1000)
    max_depth = max_depth, # Profundidad máxima del árbol [3, 12]
    num_leaves = 120, # Número de hojas máximo del árbol, <2^max_depth
    min_child_samples = 10, # Número mínimo de registros en una hoja
    bagging_fraction = 1.0, # Porcentaje de datos cogidos en cada re-sampling del Bagging
    bagging_freq = 0, # Frecuencia del Bagging
    feature_fraction = 0.8, # Feature sub-sampling
    reg_sqrt = True, # Raíz cuadrada del target, para variables a predecir con mucha variabilidad
    feature_pre_filter = False,
    random_state = seed,
    n_jobs = -1
)

# Pipeline
model_pipeline = Pipeline([
        ("encoder", encoder),
        ("mlgb", mlgb)
    ])

In [8]:
# Medir tiempo
tiempo_inicial = time()

# Entrenar modelo
model = model_pipeline.fit(train[features], train[target])

# Medir tiempo
tiempo_final = time()
tiempo_ejecucion = tiempo_final - tiempo_inicial

print('')
print(f'El tiempo de ejecución es de {tiempo_ejecucion/60} minutos')

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0

El tiempo de ejecución es de 1.271471385161082 minutos


In [9]:
# Cálculo de métricas
train_y_real = train[target]
train_y_pred = model.predict(train[features])

test_y_real = test[target]
test_y_pred = model.predict(test[features])

metric_test_r2 = round(r2_score(test_y_real, test_y_pred), 3)
metric_adj_test_r2 = round(1-(1-metric_test_r2)*(len(test)-1)/(len(test)-len(features)-1), 3)

metric_train_mae = round(mean_absolute_error(train_y_real, train_y_pred), 3)
metric_test_mae = round(mean_absolute_error(test_y_real, test_y_pred), 3)

metric_train_rmse = round(mean_squared_error(train_y_real, train_y_pred, squared=False), 3)
metric_test_rmse = round(mean_squared_error(test_y_real, test_y_pred, squared=False), 3)

metric_train_score = round(1.00 - (metric_train_mae / train_y_real.mean()), 3)
metric_test_score = round(1.00 - (metric_test_mae / test_y_real.mean()), 3)


# Mostrar
print('R2 Score')
print(50*'-')
print(f'Test R2: {metric_test_r2}')
print(f'Test R2 Adj: {metric_adj_test_r2}')
print('')

print('Score')
print(50*'-')
print(f'Train: {metric_train_score}')
print(f'Test: {metric_test_score}')
print('')

print('MAE')
print(50*'-')
print(f'Train: {metric_train_mae}')
print(f'Test: {metric_test_mae}')
print('')

print('RMSE')
print(50*'-')
print(f'Train: {metric_train_rmse}')
print(f'Test: {metric_test_rmse}')
print('')

R2 Score
--------------------------------------------------
Test R2: 0.965
Test R2 Adj: 0.965

Score
--------------------------------------------------
Train: 0.873
Test: 0.807

MAE
--------------------------------------------------
Train: 2.192
Test: 3.207

RMSE
--------------------------------------------------
Train: 11.159
Test: 23.61



In [10]:
feature_imp = pd.DataFrame(sorted(zip(features, model[1].feature_importances_)), columns=['Feature', 'Value'])
feature_imp = feature_imp.sort_values(by=['Value'], ascending=False).reset_index(drop=True)
feature_imp

,Feature,Value
0,ID_Provincia,10254
1,Sell_In_1,9142
2,Sell_In_PP,7188
3,Marketing_PLV,7018
4,IPRI,5967
5,Tarifa_Eur_Hl,5939
6,Trade_Marketing,5885
7,Marketing_Nacional,5651
8,Mes,5497
9,Margen_Distribuidor,5302


# 2. Explicabilidad

In [11]:
df.head()

,Epoca_Covid,Mes,Tipo_Cliente,Area_Dist,ID_Provincia,Gama,Formato_1,Formato_2,Num_PdV_CI,Num_PdV_RU,...,Tarifa_Eur_Hl,Margen_Distribuidor,Egatur,Familitur,IPC,IPRI,Temperatura_Media,Precipitacion,Sell_In_1,Sell_In
0,pre-covid,10,DHC,ZZ,30,AL,No Retornable,Tercio SR,0.0,0.0,...,364.32,0.0,81.800003,0.0,1.4,102.449997,20.379999,27.65,0.32,0.32
1,pre-covid,10,DHC,ZZ,30,IN,No Retornable,Tercio SR,0.0,0.0,...,395.83,0.0,81.800003,0.0,1.4,102.449997,20.379999,27.65,0.00,0.00
2,pre-covid,10,DHC,ZZ,30,MH,Barril,Barril,0.0,0.0,...,230.36,0.0,81.800003,0.0,1.4,102.449997,20.379999,27.65,3.00,3.00
3,pre-covid,10,DHC,ZZ,30,MH,No Retornable,Quinto SR,0.0,0.0,...,268.00,0.0,81.800003,0.0,1.4,102.449997,20.379999,27.65,0.06,0.06
4,pre-covid,10,DHC,ZZ,30,MH,No Retornable,Tercio SR,0.0,0.0,...,379.42,0.0,81.800003,0.0,1.4,102.449997,20.379999,27.65,0.16,0.16


In [15]:
# Predicciones de los primero 1000 registros
# -------------------------------------------------------------------------
df_pd = df.head(1000)
predictions = pd.Series(model.predict(df_pd[features]))

In [19]:
model_pipeline

Pipeline(steps=[('encoder',
                 OrdinalEncoder(cols=[], handle_unknown='ignore', mapping=[])),
                ('mlgb',
                 LGBMRegressor(bagging_fraction=1.0, bagging_freq=0,
                               feature_fraction=0.8, feature_pre_filter=False,
                               max_depth=8, min_child_samples=10,
                               n_estimators=1000, num_leaves=120,
                               random_state=123, reg_sqrt=True))])

In [20]:
# Explicabilidad
# -------------------------------------------------------------------------
xpl = SmartExplainer(
    model=model_pipeline[1],
    preprocessing=model_pipeline[0]
    )

xpl.compile(
    x=df_pd[features],
    y_pred=predictions
    )

In [22]:
df_xai = xpl.to_pandas(max_contrib=45)

In [23]:
df_xai

,y_pred,feature_1,value_1,contribution_1,feature_2,value_2,contribution_2,feature_3,value_3,contribution_3,...,contribution_24,feature_25,value_25,contribution_25,feature_26,value_26,contribution_26,feature_27,value_27,contribution_27
0,0.425691,Sell_In_1,0.32,-0.654383,Formato_2,Tercio SR,-0.119972,Marketing_PLV,1.32,0.117578,...,0.002421,Num_PdV_RU,0.0,-0.001261,Tarifa_Eur_Hl,364.32,0.001065,Epoca_Covid,pre-covid,-0.000635
1,-0.000107,Sell_In_1,0.0,-0.697307,Trade_Marketing,0.0,-0.235002,Marketing_PLV,0.0,-0.204957,...,0.003082,Num_PdV_RU,0.0,-0.002517,Num_PdV_TU,0.0,-0.002401,Precipitacion,27.65,0.000744
2,4.960099,Marketing_PLV,12.95,0.901966,Trade_Marketing,-0.17,-0.303849,Marketing_Nacional,-1.14,-0.180719,...,-0.003893,Temperatura_Media,20.379999,0.002076,Precipitacion,27.65,-0.000502,Num_PdV_CI,0.0,-0.000337
3,0.132849,Sell_In_1,0.06,-0.748907,Trade_Marketing,0.0,-0.271219,Sell_In_PP,0.0,-0.106347,...,-0.0035,Num_PdV_RU,0.0,-0.003231,Area_Dist,ZZ,0.002848,Precipitacion,27.65,0.001251
4,0.482344,Sell_In_1,0.16,-0.719751,Marketing_PLV,7.62,0.49268,Trade_Marketing,0.01,-0.234523,...,-0.003939,Precipitacion,27.65,0.002497,Area_Dist,ZZ,-0.000559,ID_Provincia,30,-0.000295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,15.087449,Marketing_PLV,26.86,1.013113,Sell_In_1,11.7,0.789189,Trade_Marketing,25.53,0.327578,...,0.004862,IPC,1.5,-0.004203,Num_PdV_TU,0.0,-0.002768,Temperatura_Media,18.82,0.000959
996,2.480811,Marketing_PLV,4.95,0.513511,Sell_In_1,2.16,-0.354616,Sell_In_PP,0.0,-0.102753,...,-0.002646,Temperatura_Media,18.82,0.002173,Area_Dist,Y4,0.000665,Num_PdV_TU,0.0,0.000444
997,-0.111789,Sell_In_1,-0.08,-0.591842,Marketing_PLV,-0.06,-0.445546,Trade_Marketing,-0.2,-0.297649,...,-0.001434,ID_Provincia,06,-0.000734,Temperatura_Media,18.82,0.000713,Formato_1,No Retornable,0.000151
998,2.871024,Marketing_PLV,5.94,0.572188,Sell_In_1,3.0,-0.256442,Sell_In_PP,0.0,-0.105165,...,-0.005212,Temperatura_Media,18.82,0.004176,Precipitacion,57.669998,0.002967,Num_PdV_TU,0.0,0.000501
